# 1  -  Geração de Funções

## 1.1 - Importação de Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.pipeline import Pipeline


## 1.2 - Carregamento de Dados

In [5]:
# Função para carregar dados de um arquivo CSV
def load_data(file_path):
    return pd.read_csv(file_path)


## 1.3 - Pré-processamento de Dados

In [6]:

def preprocess_training_data(df, columns_to_remove=None, fit=True):
    if columns_to_remove is None:
        columns_to_remove = [
            'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
            'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 
            'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login'
        ]

    df.drop(columns=columns_to_remove, axis=1, errors='ignore', inplace=True)
    df.dropna(inplace=True)

    if 'class' in df.columns:
        class_column = df.pop('class')  # Remove 'class' temporariamente
    else:
        class_column = None

    categorical_cols = df.select_dtypes(include=['object']).columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns

    # Adiciona a coluna 'class' de volta ao DataFrame
    if class_column is not None:
        df['class'] = class_column.values

    return df, encoder, imputer

def preprocess_new_data(df, encoder, imputer, scaler, numeric_cols):
    df.drop(['src_ip', 'dst_ip', 'timestamp', 'src_port', 'dst_port', 'frame_length'], axis=1, errors='ignore', inplace=True)

    if 'class' in df.columns:
        class_column = df.pop('class')  # Remove 'class' temporariamente
    else:
        class_column = None

    categorical_cols = df.select_dtypes(include=['object']).columns

    # Preencher valores ausentes usando o imputer ajustado
    df[numeric_cols] = imputer.transform(df[numeric_cols])

    # Codificação One-Hot com o encoder ajustado
    encoded_categorical = pd.DataFrame(
        encoder.transform(df[categorical_cols]),
        columns=encoder.get_feature_names_out(categorical_cols),
        index=df.index
    )

    df.drop(columns=categorical_cols, inplace=True)
    df = pd.concat([df, encoded_categorical], axis=1)

    # Escalonamento com o scaler ajustado
    df[numeric_cols] = scaler.transform(df[numeric_cols])

    # Adiciona a coluna 'class' de volta ao DataFrame
    if class_column is not None:
        df['class'] = class_column.values

    return df

Coluna 'class' restaurada com ['normal' 'anomaly'] valores exclusivos.


## 1.4 - Criação do Pipeline

In [ ]:
def create_pipeline(X_train):
    numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
    categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', RobustScaler())
    ])

    categorical_transformer = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ]
    )

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(random_state=42))
    ])

    return pipeline

## 1.5 - Divisão e Balanceamento dos Dados

In [7]:
# Função para dividir e balancear os dados usando SMOTE
def split_and_balance_data(df, method='smote'):
    X = df.drop(['class'], axis=1)
    y = df['class']
    if method == 'smote':
        oversampler = SMOTE(random_state=42)
    X_resampled, y_resampled = oversampler.fit_resample(X, y)
    return train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

## 1.6 - Alinhamento de Colunas

In [8]:
def align_and_add_missing_columns(df, reference_columns):
    missing_cols = set(reference_columns) - set(df.columns)
    for col in missing_cols:
        df[col] = 0
    df = df[reference_columns]
    return df


## 1.7 - Otimização de Hiperparâmetros do Modelo

In [ ]:
# Função para otimizar um modelo usando pipeline e busca em grade
def optimize_model_with_pipeline(X_train, y_train):
    pipeline = create_pipeline(X_train)
    param_grid = {
        'classifier__n_estimators': [50, 100, 150],
        'classifier__max_features': ['sqrt', 'log2'],
        'classifier__max_depth': [None, 10, 20, 30],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__min_samples_leaf': [1, 2, 4]
    }
     # Configura a busca em grade com validação cruzada (cv=5) e acurácia como métrica
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    print(f'Melhores hiperparâmetros: {grid_search.best_params_}')
    return grid_search.best_estimator_


## 1.8 - Treinamento com Validação Cruzada

In [ ]:
# treina o modelo com validação cruzada
def train_model_with_cross_validation(X_train, y_train):
    rf_classifier = RandomForestClassifier(random_state=42, class_weight='balanced')
     
    # Utiliza StratifiedKFold para cross-validation para dividir os dados em partes
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    cv_scores = cross_val_score(rf_classifier, X_train, y_train, cv=skf, scoring='accuracy')
    print(f'Pontuações de cada fold: {cv_scores}')
    print(f'Média da Acurácia: {np.mean(cv_scores)}')
    print(f'Desvio Padrão da Acurácia: {np.std(cv_scores)}')
    rf_classifier.fit(X_train, y_train)
    return rf_classifier


## 1.9 - Avaliação do Modelo

In [ ]:
def evaluate_model(model, X, y):
    predictions = model.predict(X)
    accuracy = accuracy_score(y, predictions)
    precision = precision_score(y, predictions, average='weighted', zero_division=0)
    recall = recall_score(y, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y, predictions, average='weighted', zero_division=0)
    return accuracy, precision, recall, f1, predictions

def plot_confusion_matrix(y_true, y_pred, labels):
    # Gera a matriz de confusão
    cm = confusion_matrix(y_true, y_pred)
    print(cm)

    cm = cm.astype('int')  # Converte para inteiros para garantir o formato correto
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels, cbar=False)

    plt.title('Matriz de detecção de intrusões')
    plt.xlabel('Previstos')
    plt.ylabel('Reais')
    plt.show()


## 1.10 - Geração da Curva de Aprendizado 

In [ ]:
def plot_learning_curve(estimator, X, y, cv=5, scoring='accuracy', train_sizes=np.linspace(0.1, 1.0, 10)):
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, scoring=scoring, train_sizes=train_sizes, random_state=42, n_jobs=-1
    )
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    plt.figure()
    plt.title("Curva de Aprendizado")
    plt.xlabel("Tamanho do Conjunto de Treinamento")
    plt.ylabel(scoring.capitalize())
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="r")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Pontuação de Treinamento")

    plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Pontuação de Validação")

    plt.legend(loc="best")
    plt.show()


# 2 - Realização do Algoritmo Random Forest

## 2.1 - Carregamento e pré-processamento dos dados de treinamento

In [ ]:
# Carregar e pré-processar os dados de treinamento
df_train = load_data('Train_data.csv')
df_train, encoder, imputer, scaler, numeric_cols = preprocess_training_data(df_train)

## 2.2 - Divisão de dados em conjuntos de treinamento e teste

In [ ]:
X_train_val, X_test, y_train_val, y_test = split_and_balance_data(df_train, method='smote')
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

## 2.3 - Criação e Ajuste do pipeline

In [ ]:
# Criar o pipeline
pipeline = create_pipeline(X_train)

# Ajustar o pipeline com os dados de treinamento
pipeline.fit(X_train, y_train)


## 2.4 - Geração de Curvas de aprendizado utilizando validação cruzada

In [ ]:
# Otimizar e treinar o modelo usando o GridSearchCV no pipeline
optimized_model = pipeline(X_train, y_train)


# Ajustar e treinar o modelo usando validação cruzada estratificada
trained_model = train_model_with_cross_validation(X_train, y_train)

# Gerar curva de aprendizado
plot_learning_curve(pipeline, X_train, y_train)

## 2.5 - Avaliação do modelo no conjunto de teste

In [ ]:
# Avaliar o modelo no conjunto de teste
test_accuracy, test_precision, test_recall, test_f1, test_predictions = evaluate_model(pipeline, X_test, y_test)
print("Métricas de avaliação no conjunto de teste:")
print("Acurácia:", test_accuracy)
print("Precisão:", test_precision)
print("Recall:", test_recall)
print("F1-score:", test_f1)


## 2.6 - Geração dos ataques DoS

In [ ]:
# Realização dos ataques Dos 
hping3 10.117.17.19 -S -p 80 --flood

### 2.6.0 - Extração de informações dos pacotes de ataque capturados

In [ ]:
tshark -r "novos-ataques-dos.pcap"
-T fields -E header=y -E separator=, -E quote=d -E occurrence=f
-e ip.src -e ip.dst -e ip.proto -e ip.checksum -e tcp.srcport -e tcp.dstport -e tcp.flags -e frame.len -e frame.time_epoch
> novos-ataques-dos.csv

### 2.6.1 - Importação e Leitura dos dados extraídos

In [ ]:
from scapy.all import sniff, IP, TCP
dados_extraidos = pd.read_csv('novo-ataques-dos.csv')
dados_extraidos

### 2.6.2 -  Definição dos Dicionários de Mapeamento

In [ ]:
# Mapeamento entre números de protocolo e seus respectivos nomes
protocol_mapping = {
    1: "icmp",
    6: "tcp",
    17: "udp"
}

# Mapeamento de portas para serviços
port_to_service = {
    20: "ftp",
    21: "ftp",
    22: "ssh",
    23: "telnet",
    25: "smtp",
    53: "domain",
    80: "http",
    110: "pop3",
    143: "imap",
    443: "https",
}

### 2.6.3 - Definição de Funções Auxiliares

In [ ]:
# Função para converter o número do protocolo em texto
def convert_protocol(proto_number):
    return protocol_mapping.get(proto_number, "Unknown")

# Função para determinar o serviço com base na porta
def get_service(port):
    return port_to_service.get(port, "other")

# Função para inferir flags baseado nos dados disponíveis
def infer_flag(df):
    flags = []
    for _, row in df.iterrows():
        if row['protocol_type'] == "tcp":
            if row['src_port'] < row['dst_port']:
                flags.append('SF')
            else:
                flags.append('S0')
        else:
            flags.append('OTH')
    return flags

# Função para inferir se é um ataque do tipo land
def infer_land(df):
    lands = []
    for _, row in df.iterrows():
        if row['src_ip'] == row['dst_ip']:
            lands.append(1)
        else:
            lands.append(0)
    return lands

# Função para inferir fragmentos errados
def infer_wrong_fragment(df):
    wrong_fragments = []
    for _, row in df.iterrows():
        if 'ip.flags.df' in df.columns and row['ip.flags.df'] == 0 and row['ip.frag_offset'] != 0:
            wrong_fragments.append(1)
        else:
            wrong_fragments.append(0)
    return wrong_fragments


# Função para inferir a coluna 'urgent'
def infer_urgent(df):
    urgents = []
    for _, row in df.iterrows():
        if 'tcp.flags.urg' in df.columns:
            urgents.append(row['tcp.flags.urg'])
        else:
            urgents.append(0)
    return urgents


### 2.6.4 - Cálculo de Métricas por Conexão

In [ ]:
# Função para calcular 'count' e 'srv_count'
def calculate_counts(df):
    df['count'] = df.apply(lambda row: ((df['dst_ip'] == row['dst_ip']) & 
                                        (df['timestamp'] <= row['timestamp']) & 
                                        (df['timestamp'] > row['timestamp'] - pd.Timedelta(seconds=2))).sum(), axis=1)
    
    df['srv_count'] = df.apply(lambda row: ((df['dst_port'] == row['dst_port']) & 
                                            (df['timestamp'] <= row['timestamp']) & 
                                            (df['timestamp'] > row['timestamp'] - pd.Timedelta(seconds=2))).sum(), axis=1)
    return df

### 2.6.5 - Cálculo de Taxas de Erro

In [ ]:
# Função para calcular Serror Rate e Srv Serror Rate
def calculate_serror_rates(df):
    df['serror_rate'] = df.apply(lambda row: ((df['dst_ip'] == row['dst_ip']) & 
                                              (df['flag'] == 'S0')).sum() / row['count'] if row['count'] > 0 else 0, axis=1)
    
    df['srv_serror_rate'] = df.apply(lambda row: ((df['dst_port'] == row['dst_port']) & 
                                                  (df['flag'] == 'S0')).sum() / row['srv_count'] if row['srv_count'] > 0 else 0, axis=1)
    return df

# Função para calcular Rerror Rate e Srv Rerror Rate
def calculate_rerror_rates(df):
    df['rerror_rate'] = df.apply(lambda row: ((df['dst_ip'] == row['dst_ip']) & 
                                              (df['flag'] == 'REJ')).sum() / row['count'] if row['count'] > 0 else 0, axis=1)
    
    df['srv_rerror_rate'] = df.apply(lambda row: ((df['dst_port'] == row['dst_port']) & 
                                                  (df['flag'] == 'REJ')).sum() / row['srv_count'] if row['srv_count'] > 0 else 0, axis=1)
    return df

### 2.6.6 - Cálculo de Taxas de Serviço

In [ ]:
# Função para calcular Same Srv Rate e Diff Srv Rate
def calculate_srv_rates(df):
    df['same_srv_rate'] = df.apply(lambda row: ((df['dst_ip'] == row['dst_ip']) & 
                                                (df['service'] == row['service'])).sum() / row['count'] if row['count'] > 0 else 0, axis=1)
    
    df['diff_srv_rate'] = df.apply(lambda row: ((df['dst_ip'] == row['dst_ip']) & 
                                                (df['service'] != row['service'])).sum() / row['count'] if row['count'] > 0 else 0, axis=1)
    return df

# Função para calcular Srv Diff Host Rate
def calculate_srv_diff_host_rate(df):
    df['srv_diff_host_rate'] = df.apply(lambda row: ((df['dst_ip'] != row['dst_ip']) & 
                                                     (df['service'] == row['service'])).sum() / row['srv_count'] if row['srv_count'] > 0 else 0, axis=1)
    return df

### 2.6.7 - Carregamento e Processamento dos Dados

In [ ]:
# Carregar o CSV extraído
df = pd.read_csv("C:\\Users\\igorm\\Downloads\\Implementacao-TCC\\novos-ataques-dos.csv")

# Converter frame.time_epoch para datetime
df['frame.time_epoch'] = pd.to_datetime(df['frame.time_epoch'], unit='s')

# Converter o campo ip.proto para texto
df['ip.proto'] = df['ip.proto'].apply(convert_protocol)

# Calcular a duração, src_bytes e dst_bytes por conexão individual
df['duration'] = df.groupby(['ip.src', 'ip.dst', 'ip.proto'])['frame.time_epoch'].transform(lambda x: (x.max() - x.min()).total_seconds())
df['src_bytes'] = df.groupby(['ip.src', 'ip.dst', 'ip.proto'])['frame.len'].transform('sum')
df['dst_bytes'] = df.groupby(['ip.dst', 'ip.src', 'ip.proto'])['frame.len'].transform('sum')

# Adicionar a coluna 'service' baseada na porta de destino
df['service'] = df['tcp.dstport'].apply(get_service)

### 2.6.8 - Cálculo das Métricas do Host de Destino 

In [ ]:
# Função para calcular as métricas Dst Host
def calculate_dst_host_metrics(df):
    df['dst_host_count'] = df.groupby('dst_ip')['dst_ip'].transform('count')
    df['dst_host_srv_count'] = df.groupby(['dst_ip', 'service'])['service'].transform('count')
    df['dst_host_same_srv_rate'] = df['dst_host_srv_count'] / df['dst_host_count']
    df['dst_host_diff_srv_rate'] = df.groupby('dst_ip')['service'].transform(lambda x: x.nunique()) / df['dst_host_count']
    df['dst_host_same_src_port_rate'] = df.groupby('dst_ip')['src_port'].transform(lambda x: x.nunique()) / df['dst_host_count']
    df['dst_host_srv_diff_host_rate'] = df.groupby('service')['dst_ip'].transform(lambda x: x.nunique()) / df['dst_host_count']
    df['dst_host_serror_rate'] = df.groupby('dst_ip')['flag'].transform(lambda x: (x == 'S0').sum()) / df['dst_host_count']
    df['dst_host_srv_serror_rate'] = df.groupby(['dst_ip', 'service'])['flag'].transform(lambda x: (x == 'S0').sum()) / df['dst_host_srv_count']
    df['dst_host_rerror_rate'] = df.groupby('dst_ip')['flag'].transform(lambda x: (x == 'REJ').sum()) / df['dst_host_count']
    df['dst_host_srv_rerror_rate'] = df.groupby(['dst_ip', 'service'])['flag'].transform(lambda x: (x == 'REJ').sum()) / df['dst_host_srv_count']
    return df

### 2.6.9 - Renomeação das colunas 

In [ ]:
# Renomear colunas para corresponder aos nomes NSL-KDD
df.rename(columns={
    'ip.src': 'src_ip',
    'ip.dst': 'dst_ip',
    'ip.proto': 'protocol_type',
    'tcp.srcport': 'src_port',
    'tcp.dstport': 'dst_port',
    'frame.len': 'frame_length',
    'frame.time_epoch': 'timestamp'
}, inplace=True)


### 2.6.10 - Realização dos cálculos

In [ ]:

# Inferir a coluna 'flag' após renomear as colunas
df['flag'] = infer_flag(df)

# Inferir a coluna 'land'
df['land'] = infer_land(df)

# Inferir a coluna 'wrong_fragment'
df['wrong_fragment'] = infer_wrong_fragment(df)

# Inferir a coluna 'urgent'
df['urgent'] = infer_urgent(df)

## Calcular 'count' e 'srv_count'
df = calculate_counts(df)

# Calcular Serror Rate e Srv Serror Rate
df = calculate_serror_rates(df)

# Calcular Rerror Rate e Srv Rerror Rate
df = calculate_rerror_rates(df)

# Calcular Same Srv Rate e Diff Srv Rate
df = calculate_srv_rates(df)

# Calcular Srv Diff Host Rate
df = calculate_srv_diff_host_rate(df)

# Calcular métricas específicas do host de destino
df = calculate_dst_host_metrics(df)


### 2.6.11 - Salvamento e Exibição

In [ ]:
# Selecionar apenas as colunas necessárias
df = df[['src_ip', 'dst_ip', 'protocol_type', 'src_port', 'dst_port', 'duration', 'src_bytes', 'dst_bytes', 'service', 'flag']]

# Salvar o CSV processado
df.to_csv("novos-ataques-dos.csv", index=False)

# Apresentar os dados na tela
print(df)

### 2.6.12 - Definição dos IPs considerados como anomalias

In [ ]:
import pandas as pd
import random
import ipaddress

# Função para gerar um IP aleatório
def random_ip():
    return str(ipaddress.IPv4Address(random.randint(0, (1 << 32) - 1)))

# Caminho do arquivo CSV original
csv_file = 'novos-ataques-dos.csv'

# Carregar o arquivo CSV em um DataFrame do pandas
df = pd.read_csv(csv_file)

# IPs alvo a serem classificados como anomalias
target_ip_1 = "10.117.77.119"
target_ip_2 = "10.117.77.120"

# Garantir que todos os valores da coluna 'ip.src' sejam strings
df['ip.src'] = df['ip.src'].astype(str)

# Adicionar coluna 'class' e modificar IPs de origem
df['class'] = df['ip.src'].apply(lambda x: 'anomaly' if x == target_ip_1 or x == target_ip_2 else 'normal')
df['ip.src'] = df.apply(lambda row: random_ip() if row['ip.src'] == target_ip_1 else row['ip.src'], axis=1)

# Caminho do novo arquivo CSV
output_csv_file = 'dados-coletados.csv'

# Salvar o DataFrame modificado em um novo arquivo CSV
df.to_csv(output_csv_file, index=False)

print("Arquivo CSV modificado gerado com sucesso.")


## 2.7 - Carregamento e pré-processamento do novo conjunto de dados

In [ ]:
# Carregar o novo conjunto de dados
df_new = load_data("dados-coletados.csv")

# Fazer o pré-processamento nos dados novos
df_new = preprocess_new_data(df_new, encoder=encoder, imputer=imputer, scaler=scaler, numeric_cols=numeric_cols)


## 2.8 - Adição e alinhamento das colunas 

In [ ]:

# Verificação de presença da coluna 'class' após o pré-processamento
if 'class' not in df_new.columns:
    print("A coluna 'class' não foi encontrada após o pré-processamento.")


if 'class' in df_new.columns:
    y_new = df_new['class']
    X_new = df_new.drop(['class'], axis=1)
    
    X_new = align_and_add_missing_columns(X_new, reference_columns=X_train_val.columns)


## 2.9 - Realização e Avaliação de previsões dos novos dados 

In [ ]:

# Fazer previsões nos dados novos
    new_predictions = pipeline.predict(df_new)
    
    df_new['predictions'] = new_predictions
    
    accuracy, precision, recall, f1, new_predictions = evaluate_model(pipeline, X_new, y_new)
    print("Métricas de avaliação no novo conjunto de dados:")
    print("Acurácia:", accuracy)
    print("Precisão:", precision)
    print("Recall:", recall)
    print("F1-score:", f1)
    plot_confusion_matrix(y_new, new_predictions, labels=np.unique(y_new))
else:
    df_new = align_and_add_missing_columns(df_new, reference_columns=X_train_val.columns)
    
  
    print("Predições no novo conjunto de dados (sem rótulos verdadeiros):")
    print(df_new['predictions'].value_counts())

# Exibir mais linhas do DataFrame com previsões e métricas
print(df_new.head())

# Salvar o DataFrame com previsões e métricas em um novo arquivo CSV
try:
    df_new.to_csv("dados-finais-previstos.csv", index=False)
    print(f"As previsões e métricas foram geradas e salvas.")
except Exception as e:
    print(f"Ocorreu um erro ao salvar o arquivo: {e}")